In [2]:
#https://spark.apache.org/docs/latest/sql-data-sources-csv.html
#https://sparkbyexamples.com/pyspark/pyspark-convert-array-column-to-string-column/
#! /usr/bin/python
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[4]").appName("air_quality_pyspark").getOrCreate()

df = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/Data_Hourly_2018.csv")
df19 = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/Data_Hourly_2019.csv")
df20 = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/Data_Hourly_2020.csv")
df21 = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/Data_Hourly_2021.csv")
df= df.union(df19).union(df20).union(df21)
df.show()
#df = df.withColumn("date_time",to_timestamp(df.date_time,'dd/MM/yyyy HH:mm'))
#df = df.withColumn("date",date_format(df.date_time,'yyyy-MM-dd'))

#df = df.withColumn("pollutant_value",df.pollutant_value.cast(DoubleType()))
#df_aggr_daily = df.groupby(['date','station_code','pollutant_id']).mean("pollutant_value").withColumnRenamed("avg(pollutant_value)","mean_pollutant_value")

#df_aggr_daily.write.option("delimiter", ",").option("header", True).mode('overwrite').csv('gs://air_quality_cyprus/Data_Daily_All') 


Py4JJavaError: An error occurred while calling o28.csv.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:752)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:750)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNec

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType

df = df.withColumn("date_time",to_timestamp(df.date_time,'dd/MM/yyyy HH:mm'))
df = df.withColumn("date",date_format(df.date_time,'yyyy-MM-dd'))
df = df.withColumn("pollutant_value",df.pollutant_value.cast(DoubleType()))
df_aggr_daily=  df.groupby(['date','station_code','pollutant_id']).mean("pollutant_value").withColumnRenamed("avg(pollutant_value)","mean_pollutant_value")

df_aggr_daily.show(truncate=False) 

+----------+------------+------------+--------------------+
|date      |station_code|pollutant_id|mean_pollutant_value|
+----------+------------+------------+--------------------+
|2018-01-29|1           |45          |2.4291666666666667  |
|2018-05-24|1           |45          |0.3041666666666667  |
|2018-08-21|1           |45          |0.325               |
|2018-11-22|3           |45          |null                |
|2018-02-27|8           |45          |0.2916666666666666  |
|2018-03-18|8           |45          |0.16250000000000003 |
|2018-05-09|8           |45          |null                |
|2018-12-08|8           |45          |0.9666666666666668  |
|2018-02-12|9           |45          |0.620833333333333   |
|2018-06-25|9           |45          |0.27499999999999997 |
|2018-09-19|9           |45          |0.29166666666666663 |
|2018-11-08|9           |45          |0.5333333333333333  |
|2018-06-07|15          |45          |0.26249999999999996 |
|2018-12-31|14          |45          |0.

In [3]:
df_stations = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/AirQualityMonitoringStations.csv")
df_stations = df_stations.select(col("station_code"),col("station_name_en")) #,col("latitude"),col("longitude")
df_stations = df_stations.withColumnRenamed('station_code','station_code_pk')
df_stations.show()

+---------------+--------------------+
|station_code_pk|     station_name_en|
+---------------+--------------------+
|              1|Nicosia -Traffic ...|
|              2|Nicosia - Residen...|
|              3|Limassol -Traffic...|
|              5|Larnaka - Traffic...|
|              8|Zygi -Industrial ...|
|              9|EMEP- Ayia Marina...|
|             14|Mari - Industrial...|
|             15|Pafos - Traffic S...|
|             16|Paralimni - Traff...|
+---------------+--------------------+



In [4]:
df_Pollutants = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/PollutantsId.csv")
df_Pollutants = df_Pollutants.select(col("pollutant_id"),col("pollutant_code"),col("pollutant_name_en"),col("Unit_of_measurement_en"))
df_Pollutants = df_Pollutants.withColumnRenamed('pollutant_id','pollutant_id_pk')
# Unit_of_measurement_en:μg/m³
df_Pollutants.show()

+---------------+--------------+--------------------+----------------------+
|pollutant_id_pk|pollutant_code|   pollutant_name_en|Unit_of_measurement_en|
+---------------+--------------+--------------------+----------------------+
|              1|            NO|      Nitrogen Oxide|                 μg/m³|
|              2|           NO2|    Nitrogen Dioxide|                 μg/m³|
|              3|           NOX|     Nitrogen Oxides|                 μg/m³|
|              4|           SO2|      Sulfur Dioxide|                 μg/m³|
|              5|            O3|              Ozone |                 μg/m³|
|              6|            CO|     Carbon Monoxide|                 μg/m³|
|             25|          PM10|Particulate Matte...|                 μg/m³|
|             26|         PM2.5|Particulate Matte...|                 μg/m³|
|             45|          C6H6|             Benzene|                 μg/m³|
+---------------+--------------+--------------------+----------------------+

In [5]:
#Join on aggr data
df_aggr_daily = df_aggr_daily.join(df_stations,df_aggr_daily.station_code == df_stations.station_code_pk, how="inner")
df_aggr_daily = df_aggr_daily.join(df_Pollutants,df_aggr_daily.pollutant_id == df_Pollutants.pollutant_id_pk, how="inner")
df_aggr_daily = df_aggr_daily.drop("station_code_pk").drop("pollutant_id_pk")
df_aggr_daily.show()

+----------+------------+------------+--------------------+--------------------+--------------+-----------------+----------------------+
|      date|station_code|pollutant_id|mean_pollutant_value|     station_name_en|pollutant_code|pollutant_name_en|Unit_of_measurement_en|
+----------+------------+------------+--------------------+--------------------+--------------+-----------------+----------------------+
|2021-08-10|           1|          45|  0.4428034583333334|Nicosia -Traffic ...|          C6H6|          Benzene|                 μg/m³|
|2021-01-16|           5|          45|  0.6404166666666667|Larnaka - Traffic...|          C6H6|          Benzene|                 μg/m³|
|2021-05-20|           5|          45|  0.5770833333333335|Larnaka - Traffic...|          C6H6|          Benzene|                 μg/m³|
|2021-05-28|           8|          45| 0.22624999999999995|Zygi -Industrial ...|          C6H6|          Benzene|                 μg/m³|
|2021-08-19|           8|          45| 0.

In [33]:
df_aggr_daily

date,station_code,pollutant_id,mean_pollutant_value,station_name_en,pollutant_code,pollutant_name_en,Unit_of_measurement_en
2018-03-14,3,4,1.1375000000000002,Limassol -Traffic...,SO2,Sulfur Dioxide,μg/m³
2018-04-21,3,4,1.629166666666667,Limassol -Traffic...,SO2,Sulfur Dioxide,μg/m³
2018-06-20,3,4,1.3541666666666667,Limassol -Traffic...,SO2,Sulfur Dioxide,μg/m³
2018-10-18,3,4,1.3208333333333335,Limassol -Traffic...,SO2,Sulfur Dioxide,μg/m³
2018-10-30,3,4,3.311111111111111,Limassol -Traffic...,SO2,Sulfur Dioxide,μg/m³
2018-12-15,3,4,1.5291666666666668,Limassol -Traffic...,SO2,Sulfur Dioxide,μg/m³
2018-03-06,5,4,4.529166666666667,Larnaka - Traffic...,SO2,Sulfur Dioxide,μg/m³
2018-06-06,5,4,4.375000000000001,Larnaka - Traffic...,SO2,Sulfur Dioxide,μg/m³
2018-10-01,8,4,0.43750000000000006,Zygi -Industrial ...,SO2,Sulfur Dioxide,μg/m³
2018-11-19,8,4,3.454166666666667,Zygi -Industrial ...,SO2,Sulfur Dioxide,μg/m³


In [32]:
df_aggr_daily.dtypes

[('date', 'string'),
 ('station_code', 'string'),
 ('pollutant_id', 'string'),
 ('mean_pollutant_value', 'double'),
 ('station_name_en', 'string'),
 ('pollutant_code', 'string'),
 ('pollutant_name_en', 'string'),
 ('Unit_of_measurement_en', 'string')]

In [1]:
!scala -version

Scala code runner version 2.12.10 -- Copyright 2002-2019, LAMP/EPFL and Lightbend, Inc.


In [7]:
spark.sparkContext._jvm.scala.util.Properties.versionString()

'version 2.12.10'

In [12]:
spark = SparkSession.builder \
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()
# spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [6]:
# https://github.com/tfayyaz/cloud-dataproc/blob/master/notebooks/python/1.2.%20BigQuery%20Storage%20%26%20Spark%20SQL%20-%20Python.ipynb
table = "comp548dl-big-data.air_quality_cyprus.daily_data"
bigquerydf = spark.read.format("bigquery").option("table", table).load();
#.option("credentialsFile","/Users/rc_user/pyspark_bq_tutorial/my-rcs-project-833123-ef45632b1b12.json") \
bigquerydf.show();

Py4JJavaError: An error occurred while calling o92.load.
: java.lang.ClassNotFoundException: Failed to find data source: bigquery. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:678)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:213)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.co

In [9]:
# Saving the data to BigQuery
df_aggr_daily.write.format('bigquery') \
  .option('table', 'comp548dl-big-data.air_quality_cyprus.daily_data_v2') \
  .save()

Py4JJavaError: An error occurred while calling o188.save.
: java.lang.ClassNotFoundException: Failed to find data source: bigquery. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:678)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:265)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnecti

In [43]:
%%bigquery
SELECT *
FROM comp548dl-big-data.air_quality_cyprus.daily_data
LIMIT 10

,date,station_code,pollutant_id,mean_pollutant_value,station_name_en,pollutant_code,pollutant_name_en,Unit_of_measurement_en
0,2018-11-20,1,45,2.020833,Nicosia -Traffic Station,C6H6,Benzene,μg/m³
1,2021-02-24,1,45,0.906571,Nicosia -Traffic Station,C6H6,Benzene,μg/m³
2,2018-06-25,1,45,0.279167,Nicosia -Traffic Station,C6H6,Benzene,μg/m³
3,2019-10-26,1,45,1.000000,Nicosia -Traffic Station,C6H6,Benzene,μg/m³
4,2019-08-09,1,45,0.395833,Nicosia -Traffic Station,C6H6,Benzene,μg/m³
5,2020-10-05,1,45,0.920833,Nicosia -Traffic Station,C6H6,Benzene,μg/m³
6,2020-11-25,1,45,2.250000,Nicosia -Traffic Station,C6H6,Benzene,μg/m³
7,2021-09-19,1,45,0.471328,Nicosia -Traffic Station,C6H6,Benzene,μg/m³
8,2021-05-31,1,45,0.279617,Nicosia -Traffic Station,C6H6,Benzene,μg/m³
9,2018-02-10,1,45,0.883333,Nicosia -Traffic Station,C6H6,Benzene,μg/m³


In [137]:
#StoreData
df_aggr_daily.write.option("delimiter", ",")\
 .option("quote", "\"").option("header", True)\
 .mode('overwrite')\
 .csv('gs://air_quality_cyprus/Data_Daily_All')

In [45]:
# DataFrames can be saved as Parquet files, maintaining the schema information.
df_aggr_daily.write.parquet("data.parquet")

# Read in the Parquet file created above.
# Parquet files are self-describing so the schema is preserved.
# The result of loading a parquet file is also a DataFrame.
parquetFile = spark.read.parquet("data.parquet")

In [49]:
parquetFile.printSchema()

root
 |-- date: string (nullable = true)
 |-- station_code: string (nullable = true)
 |-- pollutant_id: string (nullable = true)
 |-- mean_pollutant_value: double (nullable = true)
 |-- station_name_en: string (nullable = true)
 |-- pollutant_code: string (nullable = true)
 |-- pollutant_name_en: string (nullable = true)
 |-- Unit_of_measurement_en: string (nullable = true)



In [47]:
# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")
station_values = spark.sql("SELECT pollutant_id,station_code, avg(mean_pollutant_value)  FROM parquetFile Group by pollutant_id,station_code")
station_values.show()

AnalysisException: 'Table or view not found: parquetFile; line 1 pos 66'

In [85]:
#union https://stackoverflow.com/questions/37332434/concatenate-two-pyspark-dataframes
#https://sparkbyexamples.com/pyspark/pyspark-to_date-convert-timestamp-to-date/#:~:text=PySpark%20timestamp%20(%20TimestampType%20)%20consists%20of,to%20date%20on%20DataFrame%20column.
#https://sparkbyexamples.com/pyspark/pyspark-date_format-convert-date-to-string-format/

+----------------+------------+------------+---------------+-------------------+----+
|date_time       |station_code|pollutant_id|pollutant_value|date_type          |hour|
+----------------+------------+------------+---------------+-------------------+----+
|01/01/2018 0:00 |1           |45          |4.8            |2018-01-01 00:00:00|00  |
|01/01/2018 1:00 |1           |45          |5.3            |2018-01-01 01:00:00|01  |
|01/01/2018 2:00 |1           |45          |6.1            |2018-01-01 02:00:00|02  |
|01/01/2018 3:00 |1           |45          |4.0            |2018-01-01 03:00:00|03  |
|01/01/2018 4:00 |1           |45          |3.7            |2018-01-01 04:00:00|04  |
|01/01/2018 5:00 |1           |45          |1.2            |2018-01-01 05:00:00|05  |
|01/01/2018 6:00 |1           |45          |1.0            |2018-01-01 06:00:00|06  |
|01/01/2018 7:00 |1           |45          |1.1            |2018-01-01 07:00:00|07  |
|01/01/2018 8:00 |1           |45          |1.9       

Read 

In [55]:
df_readback = spark.read.option("delimiter", ",").option("header", True).csv("gs://air_quality_cyprus/Data_Daily_All")
df_readback                                                                  

DataFrame[date: string, station_code: string, pollutant_id: string, mean_pollutant_value: string]

In [56]:
df_readback.show()

+----------+------------+------------+--------------------+
|      date|station_code|pollutant_id|mean_pollutant_value|
+----------+------------+------------+--------------------+
|2018-03-11|           1|          45|  0.8571428571428571|
|2018-01-28|           3|          45|   2.895454545454545|
|2018-02-01|           3|          45|                null|
|2018-06-30|           3|          45| 0.40833333333333327|
|2018-08-09|           3|          45|                null|
|2018-10-09|           3|          45|                null|
|2018-12-02|           3|          45|                null|
|2018-12-11|           3|          45|                null|
|2018-01-24|           5|          45|  1.6124999999999998|
|2018-02-01|           9|          45|  0.7041666666666665|
|2018-09-14|          15|          45| 0.25833333333333325|
|2018-10-02|          15|          45| 0.36956521739130443|
|2018-02-03|          14|          45|                null|
|2018-02-22|          14|          45|  

### Load to Pandas for flexility and play around

In [57]:
pd_df_aggr_daily  = df_aggr_daily.toPandas()

In [67]:
pd_df_aggr_daily[pd_df_aggr_daily.isna()==True].count()

date                    0
station_code            0
pollutant_id            0
mean_pollutant_value    0
dtype: int64

In [69]:
pd_df_aggr_daily.plot()